<a href="https://colab.research.google.com/github/LEE-1017/Gener--AI/blob/main/proto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install diffusers==0.30.0 transformers accelerate safetensors --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.1 MB/s eta 0:00:00


In [ ]:
import os, json, math
from typing import List, Dict

import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display, clear_output
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "runwayml/stable-diffusion-v1-5"

# ===== 你可以調整的參數 =====
prompt   = "1girl, anime style, pink hair, glasses, soft lighting, detailed eyes"
negative = "low quality, blurry, deformed, extra fingers, watermark"

seed         = 123
H = W        = 512       # 想吃更兇就拉到 640/768/896/1024（視顯存）
total_steps  = 30
checkpoints  = [12, 24]  # 檢查點（多段人工打分）
N            = 6         # 候選數；顯存夠可調大（如 8~12）
K            = 2         # 每個檢查點後保留數
cfg          = 7.0

# 分數輸入模式："manual" 或 "file"
score_input_mode = "manual"
# score_input_mode = "file"  # 用 /content/scores_checkpoint_<step>.json

# 是否將輸入分數正規化到 [0,1] 再排序（方便未來接多來源）
normalize_scores = True

# 存檔設定
save_intermediate = True
out_dir = "/content/diffusion_prototype_outputs"
os.makedirs(out_dir, exist_ok=True)


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,   # 想要 FP32 可改成 torch.float32
    safety_checker=None
).to(device)

# 不使用 enable_attention_slicing / enable_vae_slicing / cpu_offload
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

# 你也可選擇 Euler a / DPM++ 2M Karras 等 scheduler（想換可參考 diffusers）
generator = torch.Generator(device=device).manual_seed(seed)
print("模型載入完成（無省顯存設定）")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Pl

模型載入完成（無省顯存設定）


In [ ]:
@torch.no_grad()
def encode_text(texts: List[str]):
    tok = pipe.tokenizer(
        texts,
        padding="max_length",
        max_length=pipe.tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt",
    ).to(device)
    return pipe.text_encoder(**tok).last_hidden_state

@torch.no_grad()
def decode_to_pils(latents_bchw: torch.Tensor) -> List[Image.Image]:
    imgs = pipe.vae.decode(latents_bchw / pipe.vae.config.scaling_factor).sample
    imgs = (imgs.clamp(-1, 1) + 1) / 2.0  # -> [0,1]
    imgs = imgs.detach().cpu().float().numpy()          # [B,3,H,W]
    pils = []
    for x in imgs:
        x = (np.transpose(x, (1,2,0)) * 255.0).clip(0,255).astype(np.uint8)
        pils.append(Image.fromarray(x))
    return pils

def collect_scores_manual(ids: List[int]) -> List[Dict]:
    print("請輸入每張圖的分數（越大越好；任意可比較的實數即可）")
    out = []
    for i in ids:
        s = float(input(f"  - id={i} 的分數："))
        out.append({"id": int(i), "score": float(s), "components": {}})
    return out

def collect_scores_file(step: int) -> List[Dict]:
    path = f"/content/scores_checkpoint_{step}.json"
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    for d in data:
        if "id" not in d or "score" not in d:
            raise ValueError(f"分數檔缺欄位：{d}")
    return data

def maybe_normalize_scores(scores: List[Dict]) -> List[Dict]:
    if not normalize_scores or len(scores) == 0:
        return scores
    vals = np.array([float(d["score"]) for d in scores], dtype=np.float32)
    lo, hi = np.percentile(vals, 5), np.percentile(vals, 95)
    if hi <= lo:
        vals_norm = np.ones_like(vals) * 0.5
    else:
        vals_clipped = np.clip(vals, lo, hi)
        vals_norm = (vals_clipped - lo) / (hi - lo)
    out = []
    for d, v in zip(scores, vals_norm):
        out.append({**d, "score": float(v)})
    return out

def topk_by_scores(ids: List[int], scores: List[Dict], k: int) -> List[int]:
    s_map = {int(d["id"]): float(d["score"]) for d in scores}
    ordered = sorted(ids, key=lambda x: s_map.get(int(x), -1e9), reverse=True)
    return ordered[:k]

def save_pils(pils: List[Image.Image], ids: List[int], tag: str):
    if not save_intermediate:
        return
    for im, i in zip(pils, ids):
        im.save(os.path.join(out_dir, f"{tag}_id{i}.png"))


In [ ]:
cond_emb   = encode_text([prompt] * N)
uncond_emb = encode_text([negative] * N)

latents = torch.randn(
    (N, pipe.unet.config.in_channels, H//8, W//8),
    generator=generator, device=device, dtype=torch.float16
)

pipe.scheduler.set_timesteps(total_steps, device=device)
timesteps = pipe.scheduler.timesteps

print(f"初始化完成：N={N}, checkpoints={checkpoints}, total_steps={total_steps}, size={H}x{W}")


初始化完成：N=6, checkpoints=[12, 24], total_steps=30, size=512x512


In [ ]:
@torch.no_grad()
def denoise_steps(latents, start_step_idx, end_step_idx, active_mask):
    for si in range(start_step_idx, end_step_idx):
        t = timesteps[si]
        lat_in = latents[active_mask]
        if lat_in.numel() == 0:
            break

        lat_model_in = pipe.scheduler.scale_model_input(lat_in, t)

        # classifier-free guidance
        lat_pair = torch.cat([lat_model_in, lat_model_in], dim=0)
        emb_pair = torch.cat([uncond_emb[active_mask], cond_emb[active_mask]], dim=0)

        noise_pred = pipe.unet(lat_pair, t, encoder_hidden_states=emb_pair).sample
        eps_uncond, eps_text = noise_pred.chunk(2)
        eps = eps_uncond + cfg * (eps_text - eps_uncond)

        prev = pipe.scheduler.step(eps, t, lat_in).prev_sample
        # Corrected line: Ensure assignment is done element-wise
        latents[active_mask] = prev.to(latents.dtype)
    return latents

In [ ]:
active = torch.ones(N, dtype=torch.bool, device=device)
step_cursor = 0

for cp in checkpoints + [total_steps]:

    latents = denoise_steps(latents, step_cursor, cp, active)
    step_cursor = cp
    print(cp)

    if cp == total_steps:
        break

    current_ids = [i for i, a in enumerate(active.tolist()) if a]
    pils = decode_to_pils(latents[active])

    clear_output(wait=True)
    print(f"[Checkpoint @ step {cp}] 候選數：{len(current_ids)} / {N}")
    for pid, im in zip(current_ids, pils):
        display(im)
        print(f"  ↑ id = {pid}")
    save_pils(pils, current_ids, tag=f"step{cp}")

    if score_input_mode == "manual":
        scores = collect_scores_manual(current_ids)
    else:
        print(f"從檔案讀取分數：/content/scores_checkpoint_{cp}.json")
        scores = collect_scores_file(cp)

    scores_for_rank = maybe_normalize_scores(scores)
    keep_ids = set(topk_by_scores(current_ids, scores_for_rank, K))
    print(f"保留 Top-{K}: {sorted(list(keep_ids))}")

    new_active = active.clone()
    for i in current_ids:
        if i not in keep_ids:
            new_active[i] = False
    active = new_active

print("去噪完成，解碼最終圖...")
final_ids = [i for i, a in enumerate(active.tolist()) if a]
final_pils = decode_to_pils(latents[active])

print(f"✅ 最終保留 {len(final_ids)} 個樣本：{final_ids}")
for fid, img in zip(final_ids, final_pils):
    display(img)
    img.save(os.path.join(out_dir, f"final_id{fid}.png"))
print(f"輸出已存到：{out_dir}")
